# import

In [5]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
tf.__version__

'2.3.1'

# Checkpoint Function

In [6]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

# Data load & Preprocessing Function

In [7]:
def load_mnist():
    (train_data,train_labels), (test_data,test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data,axis=-1) # [N,28,28] -> [N,28,28,1] : [batch_size,height,width,channel]
    test_data = np.expand_dims(test_data,axis=-1)   # [N,28,28] -> [N,28,28,1] axis=위치(-1=맨끝)
    
    train_data, test_data = normalize(train_data, test_data) # [0~255] -> [0~1]
    
    train_labels = to_categorical(train_labels,10) # [N,] -> [N,10]
    test_labels = to_categorical(test_labels,10) # [N,] -> [N,10] : One hot Encoding
    return train_data, train_labels, test_data, test_labels

def normalize(train_data,test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0
    return train_data, test_data

# Performance Function

In [8]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
    return loss

def accuracy_fn(model, images, labels): # 정확도
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits,-1),tf.argmax(labels,-1))
    accuracy = tf.reduce_mean(tf.cast(prediction,tf.float32))
    return accuracy

def grad(model, images, labels): # gradient
    with tf.GradientTape() as tape:
        loss = loss_fn(model,images,labels)
    return tape.gradient(loss,model.variables)

# Model Function

In [9]:
def flatten():
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init): # fully connected layer
    return tf.keras.layers.Dense(units=label_dim,use_bias=True,kernel_initializer=weight_init)
# unit = output으로 나갈 Channel이 몇개인가?
# use_bias = bias를 사용하는가 사용하지 않는가?
# kernel_initializer = weight 초기값. 

def relu():
    return tf.keras.layers.Activation(tf.keras.activations.relu)

# Create model(class version)

In [10]:
class create_model_class(tf.keras.Model):
    def __init__(self,label_dim): # label_dim : Network의 Logit을 구할 때, 최종적으로 몇개의 Output을 낼 것인가?(여기선 10)
        super(create_model_class,self).__init__()
        weight_init = tf.keras.initializers.RandomNormal() # Random Normal Initialization
        
        self.model = tf.keras.Sequential() # 층층이 쌓아나가는 것을 List처럼.
        self.model.add(flatten()) # [N,28,28,1] -> [N,784]
        # [N,784]로 바꿔주는 것은 나중에 fully connected layer를 이용할 것이기 때문. 
        # fully connected가 아니라
        
        for i in range(2):
            self.model.add(dense(256,weight_init))
            self.model.add(relu())
        
        self.model.add(dense(label_dim,weight_init))
        
    def call(self,x,training=None,mask=None):
        x = self.model(x)
        return(x)

# Create model(function version)

In [11]:
def create_model_function(label_dim):
    weight_init = tf.keras.initializers.RandomNormal()
    
    model = tf.keras.Sequential()
    model.add(flatten())
    
    for i in range(2):
        model.add(dense(256,weight_init))
        model.add(relu())
        
    model.add(dense(label_dim,weight_init))
    return model

# Define data & hyper-parameter

In [12]:
"""dataset"""
train_x, train_y, test_x, test_y = load_mnist()

"""parameters"""
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

"""Graph input using Dataset API"""
train_dataset = tf.data.Dataset.from_tensor_slices((train_x,train_y)).\
    shuffle(buffer_size = 100000).\
    prefetch(buffer_size = batch_size).\
    batch(batch_size,drop_remainder=True) # batch를 몇개만큼 네트워크에 던져 줄 것인가?
    # shuffle : 데이터셋을 섞는다. input 데이터보다 크게 설정하면 Random으로 shuffle한다.
    # prefetch : batch_size만큼 학습을 하고 있을 때, 미리 메모리에 batch_size만큼 올려놓으면 학습을 좀 더 빠르게 할 수 있다.
    # batch : batch를 몇개만큼 네트워크에 던져 줄 것인가?

test_dataset = tf.data.Dataset.from_tensor_slices((test_x,test_y)).\
    shuffle(buffer_size = 100000).\
    prefetch(buffer_size = len(test_x)).\
    batch(len(test_x))

In [13]:
"""model"""
network = create_model_function(label_dim)

"""training"""
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

"""writer"""
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_relu'

# check point의 역할?
# network가 학습을 하다가 끊겼을 때, 다시 재학습을 하고 싶은 경우, 변경되었던 Weight를 불러내는 역할.
# 혹은 Test 이미지에 대해서 정확도를 보고 싶은 경우에도 활용할 수 있음.
# 학습되어 저장된 Weight를 불러오는것.
checkpoint_dir = os.path.join(checkpoint_dir,model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir,model_dir)
logs_dir = os.path.join(logs_dir,model_dir)

# Restore Check point & Start train or test phase

In [14]:
if train_flag:
    checkpoint = tf.train.Checkpoint(dnn=network)
    
    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()
    
    # restore check-point if it exists
    could_load, checkpoint_counter = load(network, checkpoint_dir)
    
    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)
        counter = checkpoint_counter
        print('[*] Load Success')
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print('[!] Load Failed...')
    
    # train phase
    with summary_writer.as_default(): # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx,(train_input,train_label) in enumerate(train_dataset):
                grads = grad(network,train_input,train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads,network.variables))
                
                train_loss = loss_fn(network,train_input,train_label)
                train_accuracy = accuracy_fn(network,train_input,train_label)
                
                for test_input, test_label in test_dataset:
                    test_accuracy = accuracy_fn(network,test_input,test_label)
                
                tf.summary.scalar(name='train_loss',data=train_loss,step=counter)
                tf.summary.scalar(name='train_accuracy',data=train_accuracy,step=counter)
                tf.summary.scalar(name='test_accuracy',data=test_accuracy,step=counter)
                
                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_accuracy: %.4f"\
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy, test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))

# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
[!] Load Failed...
Epoch: [ 0] [    0/  468] time: 0.6223, train_loss: 2.18192816, train_accuracy: 0.2734, test_accuracy: 0.1964
Epoch: [ 0] [    1/  468] time: 0.7556, train_loss: 2.12353849, train_accuracy: 0.4844, test_accuracy: 0.3976
Epoch: [ 0] [    2/  468] time: 0.9072, train_loss: 2.06076384, train_accuracy: 0.4922, test_accuracy: 0.4866
Epoch: [ 0] [    3/  468] time: 1.0528, train_loss: 1.95668459, train_accuracy: 0.5156, test_accuracy: 0.5185
Epoch: [ 0] [    4/  468] time: 1.1974, train_loss: 1.89159453, train_accuracy: 0.6094, test_accuracy: 0.5579
Epoch: [ 0] [    5/  468] time: 1.3460, train_loss: 1.82453179, train_accuracy: 0.5781, test_accuracy: 0.5943
Epoch: [ 0] [    6/  468] time: 1.5265, train_loss: 1.63956141, train_accuracy: 0.6875, test_accuracy: 0.6171
Epoch: [ 0] [    7/  468] time: 1.6642, train_loss: 1.58453536, train_accuracy: 0.6641, test_accuracy: 0.6324
Epoch: [ 0] [    8/  468] time: 1.7968, 

Epoch: [ 0] [   74/  468] time: 10.7851, train_loss: 0.31515121, train_accuracy: 0.9219, test_accuracy: 0.9078
Epoch: [ 0] [   75/  468] time: 10.9072, train_loss: 0.34004778, train_accuracy: 0.9141, test_accuracy: 0.9055
Epoch: [ 0] [   76/  468] time: 11.0349, train_loss: 0.44394761, train_accuracy: 0.8594, test_accuracy: 0.9081
Epoch: [ 0] [   77/  468] time: 11.1678, train_loss: 0.43329227, train_accuracy: 0.8672, test_accuracy: 0.9056
Epoch: [ 0] [   78/  468] time: 11.2945, train_loss: 0.24909204, train_accuracy: 0.9062, test_accuracy: 0.9030
Epoch: [ 0] [   79/  468] time: 11.4141, train_loss: 0.25084683, train_accuracy: 0.9141, test_accuracy: 0.9033
Epoch: [ 0] [   80/  468] time: 11.5402, train_loss: 0.32169253, train_accuracy: 0.9141, test_accuracy: 0.9066
Epoch: [ 0] [   81/  468] time: 11.6847, train_loss: 0.32448494, train_accuracy: 0.8984, test_accuracy: 0.9101
Epoch: [ 0] [   82/  468] time: 11.8273, train_loss: 0.19165531, train_accuracy: 0.9297, test_accuracy: 0.9101
E

Epoch: [ 0] [  148/  468] time: 20.3999, train_loss: 0.23913473, train_accuracy: 0.9219, test_accuracy: 0.9312
Epoch: [ 0] [  149/  468] time: 20.5264, train_loss: 0.21119782, train_accuracy: 0.9453, test_accuracy: 0.9313
Epoch: [ 0] [  150/  468] time: 20.6521, train_loss: 0.26347572, train_accuracy: 0.8984, test_accuracy: 0.9328
Epoch: [ 0] [  151/  468] time: 20.7783, train_loss: 0.21826485, train_accuracy: 0.9297, test_accuracy: 0.9343
Epoch: [ 0] [  152/  468] time: 20.9030, train_loss: 0.27209184, train_accuracy: 0.9219, test_accuracy: 0.9322
Epoch: [ 0] [  153/  468] time: 21.0267, train_loss: 0.29591262, train_accuracy: 0.9219, test_accuracy: 0.9310
Epoch: [ 0] [  154/  468] time: 21.1504, train_loss: 0.19214979, train_accuracy: 0.9609, test_accuracy: 0.9293
Epoch: [ 0] [  155/  468] time: 21.2761, train_loss: 0.29798725, train_accuracy: 0.9141, test_accuracy: 0.9282
Epoch: [ 0] [  156/  468] time: 21.4018, train_loss: 0.16426954, train_accuracy: 0.9453, test_accuracy: 0.9291
E

Epoch: [ 0] [  222/  468] time: 29.9170, train_loss: 0.19547811, train_accuracy: 0.9141, test_accuracy: 0.9409
Epoch: [ 0] [  223/  468] time: 30.0427, train_loss: 0.29286811, train_accuracy: 0.9219, test_accuracy: 0.9388
Epoch: [ 0] [  224/  468] time: 30.1647, train_loss: 0.21635003, train_accuracy: 0.9375, test_accuracy: 0.9396
Epoch: [ 0] [  225/  468] time: 30.2851, train_loss: 0.16190201, train_accuracy: 0.9297, test_accuracy: 0.9378
Epoch: [ 0] [  226/  468] time: 30.4063, train_loss: 0.14681089, train_accuracy: 0.9453, test_accuracy: 0.9362
Epoch: [ 0] [  227/  468] time: 30.5313, train_loss: 0.15150487, train_accuracy: 0.9609, test_accuracy: 0.9365
Epoch: [ 0] [  228/  468] time: 30.6590, train_loss: 0.14426795, train_accuracy: 0.9844, test_accuracy: 0.9392
Epoch: [ 0] [  229/  468] time: 30.7827, train_loss: 0.22006576, train_accuracy: 0.9375, test_accuracy: 0.9416
Epoch: [ 0] [  230/  468] time: 30.9110, train_loss: 0.19894665, train_accuracy: 0.9219, test_accuracy: 0.9416
E

Epoch: [ 0] [  296/  468] time: 39.4670, train_loss: 0.21955043, train_accuracy: 0.9297, test_accuracy: 0.9513
Epoch: [ 0] [  297/  468] time: 39.6037, train_loss: 0.11463980, train_accuracy: 0.9531, test_accuracy: 0.9515
Epoch: [ 0] [  298/  468] time: 39.7385, train_loss: 0.19397280, train_accuracy: 0.9141, test_accuracy: 0.9484
Epoch: [ 0] [  299/  468] time: 39.8610, train_loss: 0.24183041, train_accuracy: 0.9453, test_accuracy: 0.9484
Epoch: [ 0] [  300/  468] time: 39.9870, train_loss: 0.23537573, train_accuracy: 0.9375, test_accuracy: 0.9464
Epoch: [ 0] [  301/  468] time: 40.1068, train_loss: 0.22840203, train_accuracy: 0.9453, test_accuracy: 0.9450
Epoch: [ 0] [  302/  468] time: 40.2314, train_loss: 0.18687044, train_accuracy: 0.9375, test_accuracy: 0.9478
Epoch: [ 0] [  303/  468] time: 40.3516, train_loss: 0.18890065, train_accuracy: 0.9375, test_accuracy: 0.9514
Epoch: [ 0] [  304/  468] time: 40.4788, train_loss: 0.20469132, train_accuracy: 0.9609, test_accuracy: 0.9527
E

Epoch: [ 0] [  370/  468] time: 49.3961, train_loss: 0.20896837, train_accuracy: 0.9219, test_accuracy: 0.9489
Epoch: [ 0] [  371/  468] time: 49.5516, train_loss: 0.23462522, train_accuracy: 0.9219, test_accuracy: 0.9508
Epoch: [ 0] [  372/  468] time: 49.7017, train_loss: 0.16992076, train_accuracy: 0.9609, test_accuracy: 0.9513
Epoch: [ 0] [  373/  468] time: 49.8476, train_loss: 0.15665743, train_accuracy: 0.9609, test_accuracy: 0.9519
Epoch: [ 0] [  374/  468] time: 49.9733, train_loss: 0.13792486, train_accuracy: 0.9375, test_accuracy: 0.9516
Epoch: [ 0] [  375/  468] time: 50.1330, train_loss: 0.09303071, train_accuracy: 0.9766, test_accuracy: 0.9535
Epoch: [ 0] [  376/  468] time: 50.2677, train_loss: 0.17670768, train_accuracy: 0.9766, test_accuracy: 0.9541
Epoch: [ 0] [  377/  468] time: 50.3897, train_loss: 0.11140739, train_accuracy: 0.9688, test_accuracy: 0.9559
Epoch: [ 0] [  378/  468] time: 50.5493, train_loss: 0.26820600, train_accuracy: 0.9062, test_accuracy: 0.9578
E

Epoch: [ 0] [  444/  468] time: 59.8533, train_loss: 0.12936364, train_accuracy: 0.9609, test_accuracy: 0.9592
Epoch: [ 0] [  445/  468] time: 59.9770, train_loss: 0.21058851, train_accuracy: 0.9531, test_accuracy: 0.9609
Epoch: [ 0] [  446/  468] time: 60.0962, train_loss: 0.12105617, train_accuracy: 0.9766, test_accuracy: 0.9607
Epoch: [ 0] [  447/  468] time: 60.2183, train_loss: 0.11988711, train_accuracy: 0.9688, test_accuracy: 0.9608
Epoch: [ 0] [  448/  468] time: 60.3410, train_loss: 0.25409603, train_accuracy: 0.9531, test_accuracy: 0.9609
Epoch: [ 0] [  449/  468] time: 60.4617, train_loss: 0.10443556, train_accuracy: 0.9609, test_accuracy: 0.9614
Epoch: [ 0] [  450/  468] time: 60.5841, train_loss: 0.08891475, train_accuracy: 0.9688, test_accuracy: 0.9612
Epoch: [ 0] [  451/  468] time: 60.7058, train_loss: 0.15990941, train_accuracy: 0.9531, test_accuracy: 0.9617
Epoch: [ 0] [  452/  468] time: 60.8292, train_loss: 0.10237323, train_accuracy: 0.9766, test_accuracy: 0.9612
E

In [15]:
# test acc = 96.26